In [113]:
import pandas as pd
import pickle

base_csv = '../data/pub_867.csv'
binary_feat = '../data/save_feats/detect_if_new_whale_bbox_xception_best_train_loss_299_feat.pkl'
HIGH_THRES = 0.95
LOW_THRES = 0.08


df = pd.read_csv(base_csv)
df.head()

,Image,Id
0,00028a005.jpg,w_dec7ffd new_whale w_f009245 w_700ebb4 w_ab44...
1,000dcf7d8.jpg,w_4132bb8 new_whale w_9d29561 w_3643be7 w_3de5...
2,000e7c7df.jpg,w_94c3bf0 new_whale w_fc6dd24 w_dcbdd01 w_3de5...
3,0019c34f4.jpg,w_a335fc2 w_c99807e new_whale w_59052ad w_0135...
4,001a4d292.jpg,new_whale w_20eeb29 w_ce66f9a w_6332313 w_5b40...


In [114]:
print(df.values[0])

['00028a005.jpg'
 'w_dec7ffd new_whale w_f009245 w_700ebb4 w_ab44ae4 w_2a0e817']


In [115]:
def keep5(s):
    return ' '.join(s.split(' ')[:5])

df.Id = df.Id.apply(keep5)
print(df.values[0])

['00028a005.jpg' 'w_dec7ffd new_whale w_f009245 w_700ebb4 w_ab44ae4']


In [116]:
with open(binary_feat,'rb') as fin:
    binary_f = pickle.load(fin)
print(binary_f.keys())
print(binary_f['pred']['00028a005.jpg'])

dict_keys(['pred', 'label_list'])
[0.18047076 0.81952924]


In [117]:
for k in binary_f['pred']:
    if binary_f['pred'][k][0]<0.08:
        print(k, binary_f['pred'][k])

01d9c00d4.jpg [0.0509856 0.9490144]
1850cbe9f.jpg [0.0659097  0.93409026]
9c97db8e1.jpg [0.07018088 0.92981917]
8d2a42bfd.jpg [0.0726124 0.9273876]
0956ac81e.jpg [0.07969202 0.920308  ]
c881bb62d.jpg [0.06342546 0.9365746 ]
704a4bf34.jpg [0.06766891 0.9323311 ]
e0b0c8163.jpg [0.06586003 0.93413997]
2913a8fda.jpg [0.0643183 0.9356817]
2fc6a2acf.jpg [0.07739871 0.92260134]
7f335301f.jpg [0.07376301 0.926237  ]
147e54767.jpg [0.06750451 0.9324955 ]
4213af5f9.jpg [0.07993134 0.9200687 ]
c3ee2fa18.jpg [0.0780101 0.9219899]
16cc3cc4b.jpg [0.06925155 0.93074846]
227d6218b.jpg [0.0797648  0.92023516]
682ddbb2f.jpg [0.07966469 0.9203353 ]
032dcd794.jpg [0.07162955 0.9283705 ]
f7663e305.jpg [0.06718195 0.93281806]
315de5250.jpg [0.07986505 0.92013496]
9fcd4102b.jpg [0.07839691 0.9216031 ]
5dd12c973.jpg [0.06558928 0.9344107 ]
0cbfdf671.jpg [0.06394118 0.9360588 ]
a04053431.jpg [0.07481499 0.92518497]
cc058fea3.jpg [0.0795221 0.920478 ]
6d3ad6d76.jpg [0.07066269 0.9293373 ]
367b03954.jpg [0.06825

In [118]:
NW = 'new_whale'

def update_label_str(label_str,p):
    tmp_list = label_str.split(' ')
    if p >= HIGH_THRES:
        if NW in label_str:
            idx = tmp_list.index(NW)
            if idx != 0:
                tmp_list[idx],tmp_list[0] = tmp_list[0],tmp_list[idx]
                return ' '.join(tmp_list)
        else:
            tmp_list = [NW] + tmp_list
            return ' '.join(tmp_list[:5])
        
    if p <= LOW_THRES:
        if NW in label_str:
            idx = tmp_list.index(NW)
            if idx != 4:
                tmp_list[idx],tmp_list[4] = tmp_list[4],tmp_list[idx]
                return ' '.join(tmp_list)
        else:
            pass
    
    return label_str

h_cnt = 0
l_cnt = 0
new_Id = []
for img_k, old_label in df.values:
    tmp_prob = binary_f['pred'][img_k]
    new_label_s = update_label_str(old_label,tmp_prob[0])
    new_Id.append(new_label_s)
    if new_label_s != old_label:
        print(old_label)
        print(new_label_s)
        print(tmp_prob)
        print('------')
        if new_label_s.startswith(NW):
            h_cnt += 1
        else:
            l_cnt += 1

w_4132bb8 new_whale w_9d29561 w_3643be7 w_3de579a
w_4132bb8 w_3de579a w_9d29561 w_3643be7 new_whale
[0.08591776 0.9140823 ]
------
w_6cda039 new_whale w_2b4b82e w_bd26110 w_ae32f8b
w_6cda039 w_ae32f8b w_2b4b82e w_bd26110 new_whale
[0.0509856 0.9490144]
------
w_9c506f6 new_whale w_5e8e218 w_f7193ed w_08630fd
w_9c506f6 w_08630fd w_5e8e218 w_f7193ed new_whale
[0.0848612 0.9151388]
------
w_a491df7 new_whale w_bb2cbcf w_350a5ba w_4488584
w_a491df7 w_4488584 w_bb2cbcf w_350a5ba new_whale
[0.07162955 0.9283705 ]
------
w_f70465f new_whale w_91b4760 w_2ef9c36 w_f13f7b3
new_whale w_f70465f w_91b4760 w_2ef9c36 w_f13f7b3
[0.91252226 0.08747774]
------
w_5557f1e new_whale w_b7411e8 w_5ad1ac9 w_17b0d3a
w_5557f1e w_17b0d3a w_b7411e8 w_5ad1ac9 new_whale
[0.08540834 0.91459167]
------
w_a3bb16b new_whale w_a47dc2d w_715c34b w_17b0d3a
new_whale w_a3bb16b w_a47dc2d w_715c34b w_17b0d3a
[0.9230139  0.07698612]
------
w_5a3e0de w_0b398b2 new_whale w_0135f5f w_9119b08
w_5a3e0de w_0b398b2 w_9119b08 w_0135f

In [119]:
print(h_cnt,l_cnt)
df.Id = new_Id
df.to_csv('../submissions/binary_up.csv',index=False)
df.head()

23 127


,Image,Id
0,00028a005.jpg,w_dec7ffd new_whale w_f009245 w_700ebb4 w_ab44ae4
1,000dcf7d8.jpg,w_4132bb8 w_3de579a w_9d29561 w_3643be7 new_whale
2,000e7c7df.jpg,w_94c3bf0 new_whale w_fc6dd24 w_dcbdd01 w_3de579a
3,0019c34f4.jpg,w_a335fc2 w_c99807e new_whale w_59052ad w_0135f5f
4,001a4d292.jpg,new_whale w_20eeb29 w_ce66f9a w_6332313 w_5b40032
